# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of each sentence from `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    source_lines = source_text.split('\n')
    print('source lines %d' % len(source_lines))
    source_ids = [[source_vocab_to_int.get(word) for word in line.split()] for line in source_lines]
    target_lines = target_text.split('\n')
    target_ids = [[target_vocab_to_int.get(word) for word in line.split()] + [target_vocab_to_int['<EOS>']] for line in target_lines]
    return source_ids, target_ids

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

source lines 4
Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

source lines 137861


# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) in [LooseVersion('1.0.0'), LooseVersion('1.0.1')], 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoding_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.

Return the placeholders in the following the tuple (Input, Targets, Learing Rate, Keep Probability)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate, keep probability)
    """
    input_placeholder = tf.placeholder(tf.int32, [None, None], name='input')
    targets_placeholder = tf.placeholder(tf.int32, [None, None])
    lr = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    return input_placeholder, targets_placeholder, lr, keep_prob

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoding Input
Implement `process_decoding_input` using TensorFlow to remove the last word id from each batch in `target_data` and concat the GO ID to the beginning of each batch.

In [8]:
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for dencoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)

    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_decoding_input(process_decoding_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer using [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn).

In [9]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :return: RNN state
    """
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size)] * num_layers)
    _, states = tf.nn.dynamic_rnn(cells, rnn_inputs, dtype=tf.float32)
    return states

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create training logits using [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder).  Apply the `output_fn` to the [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) outputs.

In [10]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param sequence_length: Sequence Length
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Train Logits
    """
    train_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, train_decoder_fn, dec_embed_input, sequence_length, scope=decoding_scope)
    return output_fn(train_pred)




"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference logits using [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder). 

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param maximum_length: The maximum allowed time steps to decode
    :param vocab_size: Size of vocabulary
    :param decoding_scope: TensorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Inference Logits
    """
    # TODO: Implement Function
    infer_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_inference(output_fn, encoder_state, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size)
    inference_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, infer_decoder_fn, scope=decoding_scope)
    return inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

- Create RNN cell for decoding using `rnn_size` and `num_layers`.
- Create the output fuction using [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to transform it's input, logits, to class logits.
- Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` function to get the training logits.
- Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob):
    """
    Create decoding layer
    :param dec_embed_input: Decoder embedded input
    :param dec_embeddings: Decoder embeddings
    :param encoder_state: The encoded state
    :param vocab_size: Size of vocabulary
    :param sequence_length: Sequence Length
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    dec_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size)] * num_layers)
    # print('vocab size: ', vocab_size, len(target_vocab_to_int))
    # print(target_vocab_to_int)
    with tf.variable_scope('decoding') as decoding_scope:
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, None, scope=decoding_scope)
        training_logits = decoding_layer_train(encoder_state, dec_cells, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)

    with tf.variable_scope("decoding", reuse=True) as decoding_scope:
        infering_logits = decoding_layer_infer(encoder_state, dec_cells, dec_embeddings, target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], sequence_length-1, vocab_size, decoding_scope, output_fn, keep_prob)


    return training_logits, infering_logits



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)`.
- Process target data using your `process_decoding_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)`.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param sequence_length: Sequence Length
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training Logits, Inference Logits)
    """
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size)
    encoder_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob)
    dec_input = process_decoding_input(target_data, target_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, dec_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    return decoding_layer(dec_embed_input, dec_embeddings, encoder_state, target_vocab_size, sequence_length,
                         rnn_size, num_layers, target_vocab_to_int, keep_prob)
    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability

In [17]:
# Number of Epochs
epochs = 30
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 13
decoding_embedding_size = 13
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.8

### Build the Graph
Build the graph using the neural network you implemented.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob = model_inputs()
    sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
        encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int)

    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import time

def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})
            
            batch_train_logits = sess.run(
                inference_logits,
                {input_data: source_batch, keep_prob: 1.0})
            batch_valid_logits = sess.run(
                inference_logits,
                {input_data: valid_source, keep_prob: 1.0})
                
            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    0/538 - Train Accuracy:  0.140, Validation Accuracy:  0.216, Loss:  5.877
Epoch   0 Batch    1/538 - Train Accuracy:  0.139, Validation Accuracy:  0.216, Loss:  5.856
Epoch   0 Batch    2/538 - Train Accuracy:  0.252, Validation Accuracy:  0.316, Loss:  5.832
Epoch   0 Batch    3/538 - Train Accuracy:  0.229, Validation Accuracy:  0.316, Loss:  5.812
Epoch   0 Batch    4/538 - Train Accuracy:  0.237, Validation Accuracy:  0.316, Loss:  5.784
Epoch   0 Batch    5/538 - Train Accuracy:  0.263, Validation Accuracy:  0.316, Loss:  5.744
Epoch   0 Batch    6/538 - Train Accuracy:  0.268, Validation Accuracy:  0.316, Loss:  5.706
Epoch   0 Batch    7/538 - Train Accuracy:  0.245, Validation Accuracy:  0.316, Loss:  5.682
Epoch   0 Batch    8/538 - Train Accuracy:  0.244, Validation Accuracy:  0.316, Loss:  5.640
Epoch   0 Batch    9/538 - Train Accuracy:  0.243, Validation Accuracy:  0.316, Loss:  5.588
Epoch   0 Batch   10/538 - Train Accuracy:  0.221, Validation Accuracy

Epoch   0 Batch   90/538 - Train Accuracy:  0.311, Validation Accuracy:  0.347, Loss:  3.237
Epoch   0 Batch   91/538 - Train Accuracy:  0.268, Validation Accuracy:  0.347, Loss:  3.353
Epoch   0 Batch   92/538 - Train Accuracy:  0.289, Validation Accuracy:  0.347, Loss:  3.319
Epoch   0 Batch   93/538 - Train Accuracy:  0.266, Validation Accuracy:  0.347, Loss:  3.340
Epoch   0 Batch   94/538 - Train Accuracy:  0.269, Validation Accuracy:  0.347, Loss:  3.358
Epoch   0 Batch   95/538 - Train Accuracy:  0.336, Validation Accuracy:  0.347, Loss:  3.088
Epoch   0 Batch   96/538 - Train Accuracy:  0.315, Validation Accuracy:  0.347, Loss:  3.160
Epoch   0 Batch   97/538 - Train Accuracy:  0.269, Validation Accuracy:  0.347, Loss:  3.307
Epoch   0 Batch   98/538 - Train Accuracy:  0.320, Validation Accuracy:  0.347, Loss:  3.144
Epoch   0 Batch   99/538 - Train Accuracy:  0.282, Validation Accuracy:  0.347, Loss:  3.291
Epoch   0 Batch  100/538 - Train Accuracy:  0.279, Validation Accuracy

Epoch   0 Batch  180/538 - Train Accuracy:  0.390, Validation Accuracy:  0.420, Loss:  2.829
Epoch   0 Batch  181/538 - Train Accuracy:  0.350, Validation Accuracy:  0.419, Loss:  2.993
Epoch   0 Batch  182/538 - Train Accuracy:  0.344, Validation Accuracy:  0.420, Loss:  3.005
Epoch   0 Batch  183/538 - Train Accuracy:  0.384, Validation Accuracy:  0.418, Loss:  2.825
Epoch   0 Batch  184/538 - Train Accuracy:  0.382, Validation Accuracy:  0.416, Loss:  2.828
Epoch   0 Batch  185/538 - Train Accuracy:  0.361, Validation Accuracy:  0.419, Loss:  2.916
Epoch   0 Batch  186/538 - Train Accuracy:  0.379, Validation Accuracy:  0.420, Loss:  2.862
Epoch   0 Batch  187/538 - Train Accuracy:  0.411, Validation Accuracy:  0.419, Loss:  2.732
Epoch   0 Batch  188/538 - Train Accuracy:  0.345, Validation Accuracy:  0.419, Loss:  2.975
Epoch   0 Batch  189/538 - Train Accuracy:  0.358, Validation Accuracy:  0.421, Loss:  2.931
Epoch   0 Batch  190/538 - Train Accuracy:  0.389, Validation Accuracy

Epoch   0 Batch  270/538 - Train Accuracy:  0.368, Validation Accuracy:  0.436, Loss:  2.741
Epoch   0 Batch  271/538 - Train Accuracy:  0.363, Validation Accuracy:  0.437, Loss:  2.739
Epoch   0 Batch  272/538 - Train Accuracy:  0.354, Validation Accuracy:  0.436, Loss:  2.786
Epoch   0 Batch  273/538 - Train Accuracy:  0.386, Validation Accuracy:  0.438, Loss:  2.672
Epoch   0 Batch  274/538 - Train Accuracy:  0.352, Validation Accuracy:  0.437, Loss:  2.784
Epoch   0 Batch  275/538 - Train Accuracy:  0.377, Validation Accuracy:  0.439, Loss:  2.704
Epoch   0 Batch  276/538 - Train Accuracy:  0.378, Validation Accuracy:  0.438, Loss:  2.712
Epoch   0 Batch  277/538 - Train Accuracy:  0.379, Validation Accuracy:  0.441, Loss:  2.669
Epoch   0 Batch  278/538 - Train Accuracy:  0.375, Validation Accuracy:  0.438, Loss:  2.663
Epoch   0 Batch  279/538 - Train Accuracy:  0.383, Validation Accuracy:  0.440, Loss:  2.667
Epoch   0 Batch  280/538 - Train Accuracy:  0.404, Validation Accuracy

Epoch   0 Batch  359/538 - Train Accuracy:  0.427, Validation Accuracy:  0.461, Loss:  2.434
Epoch   0 Batch  360/538 - Train Accuracy:  0.400, Validation Accuracy:  0.462, Loss:  2.581
Epoch   0 Batch  361/538 - Train Accuracy:  0.417, Validation Accuracy:  0.452, Loss:  2.453
Epoch   0 Batch  362/538 - Train Accuracy:  0.426, Validation Accuracy:  0.457, Loss:  2.423
Epoch   0 Batch  363/538 - Train Accuracy:  0.444, Validation Accuracy:  0.458, Loss:  2.397
Epoch   0 Batch  364/538 - Train Accuracy:  0.391, Validation Accuracy:  0.454, Loss:  2.551
Epoch   0 Batch  365/538 - Train Accuracy:  0.415, Validation Accuracy:  0.455, Loss:  2.472
Epoch   0 Batch  366/538 - Train Accuracy:  0.412, Validation Accuracy:  0.463, Loss:  2.528
Epoch   0 Batch  367/538 - Train Accuracy:  0.409, Validation Accuracy:  0.465, Loss:  2.532
Epoch   0 Batch  368/538 - Train Accuracy:  0.495, Validation Accuracy:  0.466, Loss:  2.195
Epoch   0 Batch  369/538 - Train Accuracy:  0.404, Validation Accuracy

...

Epoch  29 Batch  168/538 - Train Accuracy:  0.896, Validation Accuracy:  0.911, Loss:  0.053
Epoch  29 Batch  169/538 - Train Accuracy:  0.922, Validation Accuracy:  0.915, Loss:  0.038
Epoch  29 Batch  170/538 - Train Accuracy:  0.913, Validation Accuracy:  0.905, Loss:  0.043
Epoch  29 Batch  171/538 - Train Accuracy:  0.914, Validation Accuracy:  0.915, Loss:  0.045
Epoch  29 Batch  172/538 - Train Accuracy:  0.912, Validation Accuracy:  0.910, Loss:  0.040
Epoch  29 Batch  173/538 - Train Accuracy:  0.909, Validation Accuracy:  0.914, Loss:  0.040
Epoch  29 Batch  174/538 - Train Accuracy:  0.921, Validation Accuracy:  0.910, Loss:  0.039
Epoch  29 Batch  175/538 - Train Accuracy:  0.913, Validation Accuracy:  0.908, Loss:  0.039
Epoch  29 Batch  176/538 - Train Accuracy:  0.891, Validation Accuracy:  0.911, Loss:  0.048
Epoch  29 Batch  177/538 - Train Accuracy:  0.929, Validation Accuracy:  0.907, Loss:  0.041
Epoch  29 Batch  178/538 - Train Accuracy:  0.902, Validation Accuracy

Epoch  29 Batch  258/538 - Train Accuracy:  0.937, Validation Accuracy:  0.907, Loss:  0.040
Epoch  29 Batch  259/538 - Train Accuracy:  0.917, Validation Accuracy:  0.908, Loss:  0.041
Epoch  29 Batch  260/538 - Train Accuracy:  0.889, Validation Accuracy:  0.910, Loss:  0.047
Epoch  29 Batch  261/538 - Train Accuracy:  0.918, Validation Accuracy:  0.905, Loss:  0.049
Epoch  29 Batch  262/538 - Train Accuracy:  0.933, Validation Accuracy:  0.917, Loss:  0.043
Epoch  29 Batch  263/538 - Train Accuracy:  0.910, Validation Accuracy:  0.914, Loss:  0.042
Epoch  29 Batch  264/538 - Train Accuracy:  0.896, Validation Accuracy:  0.920, Loss:  0.049
Epoch  29 Batch  265/538 - Train Accuracy:  0.905, Validation Accuracy:  0.920, Loss:  0.052
Epoch  29 Batch  266/538 - Train Accuracy:  0.903, Validation Accuracy:  0.907, Loss:  0.048
Epoch  29 Batch  267/538 - Train Accuracy:  0.906, Validation Accuracy:  0.910, Loss:  0.045
Epoch  29 Batch  268/538 - Train Accuracy:  0.939, Validation Accuracy

Epoch  29 Batch  348/538 - Train Accuracy:  0.919, Validation Accuracy:  0.909, Loss:  0.039
Epoch  29 Batch  349/538 - Train Accuracy:  0.939, Validation Accuracy:  0.905, Loss:  0.036
Epoch  29 Batch  350/538 - Train Accuracy:  0.930, Validation Accuracy:  0.917, Loss:  0.049
Epoch  29 Batch  351/538 - Train Accuracy:  0.903, Validation Accuracy:  0.926, Loss:  0.046
Epoch  29 Batch  352/538 - Train Accuracy:  0.908, Validation Accuracy:  0.924, Loss:  0.058
Epoch  29 Batch  353/538 - Train Accuracy:  0.902, Validation Accuracy:  0.924, Loss:  0.048
Epoch  29 Batch  354/538 - Train Accuracy:  0.921, Validation Accuracy:  0.925, Loss:  0.045
Epoch  29 Batch  355/538 - Train Accuracy:  0.922, Validation Accuracy:  0.915, Loss:  0.048
Epoch  29 Batch  356/538 - Train Accuracy:  0.924, Validation Accuracy:  0.911, Loss:  0.038
Epoch  29 Batch  357/538 - Train Accuracy:  0.909, Validation Accuracy:  0.914, Loss:  0.039
Epoch  29 Batch  358/538 - Train Accuracy:  0.936, Validation Accuracy

Epoch  29 Batch  438/538 - Train Accuracy:  0.926, Validation Accuracy:  0.912, Loss:  0.037
Epoch  29 Batch  439/538 - Train Accuracy:  0.921, Validation Accuracy:  0.915, Loss:  0.043
Epoch  29 Batch  440/538 - Train Accuracy:  0.908, Validation Accuracy:  0.917, Loss:  0.043
Epoch  29 Batch  441/538 - Train Accuracy:  0.904, Validation Accuracy:  0.920, Loss:  0.052
Epoch  29 Batch  442/538 - Train Accuracy:  0.923, Validation Accuracy:  0.915, Loss:  0.035
Epoch  29 Batch  443/538 - Train Accuracy:  0.916, Validation Accuracy:  0.911, Loss:  0.044
Epoch  29 Batch  444/538 - Train Accuracy:  0.912, Validation Accuracy:  0.909, Loss:  0.045
Epoch  29 Batch  445/538 - Train Accuracy:  0.933, Validation Accuracy:  0.902, Loss:  0.035
Epoch  29 Batch  446/538 - Train Accuracy:  0.933, Validation Accuracy:  0.907, Loss:  0.041
Epoch  29 Batch  447/538 - Train Accuracy:  0.882, Validation Accuracy:  0.908, Loss:  0.045
Epoch  29 Batch  448/538 - Train Accuracy:  0.924, Validation Accuracy

Epoch  29 Batch  528/538 - Train Accuracy:  0.918, Validation Accuracy:  0.919, Loss:  0.044
Epoch  29 Batch  529/538 - Train Accuracy:  0.900, Validation Accuracy:  0.920, Loss:  0.043
Epoch  29 Batch  530/538 - Train Accuracy:  0.896, Validation Accuracy:  0.921, Loss:  0.045
Epoch  29 Batch  531/538 - Train Accuracy:  0.920, Validation Accuracy:  0.923, Loss:  0.050
Epoch  29 Batch  532/538 - Train Accuracy:  0.912, Validation Accuracy:  0.919, Loss:  0.038
Epoch  29 Batch  533/538 - Train Accuracy:  0.916, Validation Accuracy:  0.923, Loss:  0.040
Epoch  29 Batch  534/538 - Train Accuracy:  0.922, Validation Accuracy:  0.921, Loss:  0.036
Epoch  29 Batch  535/538 - Train Accuracy:  0.907, Validation Accuracy:  0.911, Loss:  0.042
Epoch  29 Batch  536/538 - Train Accuracy:  0.934, Validation Accuracy:  0.908, Loss:  0.051
Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [23]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.lower().split()]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [24]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

Input
  Word Ids:      [80, 48, 144, 106, 220, 107, 27]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [286, 89, 49, 208, 277, 10, 1]
  French Words: ['il', 'conduit', 'camion', 'jaune', 'rouillé', '.', '<EOS>']


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.